Basic Data Cleaning

from the looks of it - only study results have demographic information. So we will only be looking at those.

example of a study result
https://clinicaltrials.gov/ct2/show/results/NCT03698591?cond=TMS&draw=3&rank=7

example of no study results
https://clinicaltrials.gov/ct2/show/results/NCT04982757?cond=TMS&draw=3&rank=8

In [7]:
disease = 'rTMS'

In [8]:
from os import system
import pandas as pd
import requests
import datetime as dt
# from pytrials.client import ClinicalTrials
import json
import ipywidgets as widgets
import numpy as np
np.random.seed(10031975)
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
# from openpyxl import Workbook
import openpyxl

In [9]:
# cleaning = pd.read_excel('data/{}_cleaning.xlsx'.format(disease))
check = pd.read_csv('2-'+disease+'_cleaned_use_me.csv')

In [10]:
# cleaned.to_csv(disease+'_cleaned_use_me.csv', index = False)

In [11]:
# cleaning.head()
check.shape

(559, 37)

In [12]:
#does some error handling


import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

(2339, 35)

In [13]:
from pytrials.utils import json_handler, csv_handler


class ClinicalTrials:
    """ClinicalTrials API client

    Provides functions to easily access the ClinicalTrials.gov API
    (https://clinicaltrials.gov/api/)
    in Python.

    Attributes:
        study_fields: List of all study fields you can use in your query.
        api_info: Tuple containing the API version number and the last
        time the database was updated.
    """

    _BASE_URL = "https://clinicaltrials.gov/api/"
    _INFO = "info/"
    _QUERY = "query/"
    _JSON = "fmt=json"
    _CSV = "fmt=csv"

    def __init__(self):
        self.api_info = self.__api_info()

    @property
    def study_fields(self):
        fields_list = json_handler(
            f"{self._BASE_URL}{self._INFO}study_fields_list?{self._JSON}"
        )
        return fields_list["StudyFields"]["Fields"]

    def __api_info(self):
        """Returns information about the API"""
        last_updated = json_handler(
            f"{self._BASE_URL}{self._INFO}data_vrs?{self._JSON}"
        )["DataVrs"]
        api_version = json_handler(f"{self._BASE_URL}{self._INFO}api_vrs?{self._JSON}")[
            "APIVrs"
        ]

        return api_version, last_updated

    def get_full_studies(self, search_expr, min_rnk=1, max_studies=100):
        """Returns all content for a maximum of 100 study records.

        Retrieves information from the full studies endpoint, which gets all study fields.
        This endpoint can only output JSON (Or not-supported XML) format and does not allow
        requests for more than 100 studies at once.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.

        Returns:
            dict: Object containing the information queried with the search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 100
        """
        if max_studies > 100 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 100")

        req = f"full_studies?expr={search_expr}&min_rnk={min_rnk}&max_rnk={max_studies+min_rnk-1}&{self._JSON}"

        full_studies = json_handler(f"{self._BASE_URL}{self._QUERY}{req}")

        return full_studies

    def __repr__(self):
        return f"ClinicalTrials.gov client v{self.api_info[0]}, database last updated {self.api_info[1]}"

In [14]:
# check.head()

In [15]:
# import json

In [16]:
# print(json_string)

In [17]:
def find_values(id, json_repr):
    results = []

    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results

    # https://stackoverflow.com/questions/14048948/how-to-find-a-particular-json-value-by-key


In [18]:
##works
# NCTId = 'NCT03698591'

##no results
# NCTId = 'NCT03243214'

NCTId = 'NCT02273063'
json_temp = ct.get_full_studies(NCTId, max_studies=1, min_rnk=1)


person_json = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList'])


# person_json = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList']['BaselineMeasure'])
# json_search_str = json.dumps(person_json)

# This control the Race: ['BaselineCategory'][0]

# question = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList']['BaselineMeasure'][3]['BaselineClassList']['BaselineClass'][0]['BaselineCategoryList']['BaselineCategory'][0])


# question = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList']['BaselineMeasure'][2]['BaselineClassList']['BaselineClass'][0]['BaselineCategoryList']['BaselineCategory'][0])


# print(find_values('Description', person_json))


# Race (NIH/OMB)
# 0 - "American Indian or Alaska Native"
# 1 - "Asian"
# 2 - "Native Hawaiian or Other Pacific Islander"
# 3 - "Black or African American"
# 4 - "White"
# 5 - "More than one race"
# 6 - "Unknown or Not Reported"


# # 0 - "American Indian or Alaska Native"
# American_Indian = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList']['BaselineMeasure'][3]['BaselineClassList']['BaselineClass'][0]['BaselineCategoryList']['BaselineCategory'][0]['BaselineMeasurementList']['BaselineMeasurement'][2]['BaselineMeasurementValue'][0]) 

# # 1 - "Asian"
# Asian_total = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList']['BaselineMeasure'][3]['BaselineClassList']['BaselineClass'][0]['BaselineCategoryList']['BaselineCategory'][1]['BaselineMeasurementList']['BaselineMeasurement'][2]['BaselineMeasurementValue']) 


# # 2 - "Native Hawaiian or Other Pacific Islander"
# hawaiian_total = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList']['BaselineMeasure'][3]['BaselineClassList']['BaselineClass'][0]['BaselineCategoryList']['BaselineCategory'][2]['BaselineMeasurementList']['BaselineMeasurement'][2]['BaselineMeasurementValue']) 

# # 3 - "Black or African American"
# black_total = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList']['BaselineMeasure'][3]['BaselineClassList']['BaselineClass'][0]['BaselineCategoryList']['BaselineCategory'][3]['BaselineMeasurementList']['BaselineMeasurement'][2]['BaselineMeasurementValue']) 

# # 4 - "White"
# white_total = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList']['BaselineMeasure'][3]['BaselineClassList']['BaselineClass'][0]['BaselineCategoryList']['BaselineCategory'][4]['BaselineMeasurementList']['BaselineMeasurement'][2]['BaselineMeasurementValue']) 

# # 5 - "More than one race"
# more_than_one_total = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList']['BaselineMeasure'][3]['BaselineClassList']['BaselineClass'][0]['BaselineCategoryList']['BaselineCategory'][5]['BaselineMeasurementList']['BaselineMeasurement'][2]['BaselineMeasurementValue']) 

# # 6 - "Unknown or Not Reported"
# unknown_total = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList']['BaselineMeasure'][3]['BaselineClassList']['BaselineClass'][0]['BaselineCategoryList']['BaselineCategory'][6]['BaselineMeasurementList']['BaselineMeasurement'][2]['BaselineMeasurementValue'])

# # ['BaselineMeasureParamType']
# # ['BaselineValueList']['BaselineValue'])
# # person_json2 = json.dumps(json_temp['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselineMeasureList']['BaselineMeasure'][0])
 
# print('merican Indian or Alaska Native: ' + American_Indian)
# print('Asian: ' +Asian_total)
# print('Native Hawaiian or Other Pacific Islander: ' + hawaiian_total)
# print('Aferican American: ' + black_total)
# print('white: ' + white_total)
# print('more than one: ' + more_than_one_total)
# print('unknown: ' + unknown_total)

print(person_json)

NameError: name 'ct' is not defined

In [ ]:
import json
with open('data.json', 'w') as f:
    json.dump(person_json, f)

# person_json.to_csv('test.csv')

In [ ]:
person_json

'{"BaselineMeasureTitle": "Age, Categorical", "BaselineMeasureParamType": "Count of Participants", "BaselineMeasureUnitOfMeasure": "Participants", "BaselineClassList": {"BaselineClass": [{"BaselineCategoryList": {"BaselineCategory": [{"BaselineCategoryTitle": "<=18 years", "BaselineMeasurementList": {"BaselineMeasurement": [{"BaselineMeasurementGroupId": "BG000", "BaselineMeasurementValue": "0"}, {"BaselineMeasurementGroupId": "BG001", "BaselineMeasurementValue": "0"}, {"BaselineMeasurementGroupId": "BG002", "BaselineMeasurementValue": "0"}]}}, {"BaselineCategoryTitle": "Between 18 and 65 years", "BaselineMeasurementList": {"BaselineMeasurement": [{"BaselineMeasurementGroupId": "BG000", "BaselineMeasurementValue": "15"}, {"BaselineMeasurementGroupId": "BG001", "BaselineMeasurementValue": "15"}, {"BaselineMeasurementGroupId": "BG002", "BaselineMeasurementValue": "30"}]}}, {"BaselineCategoryTitle": ">=65 years", "BaselineMeasurementList": {"BaselineMeasurement": [{"BaselineMeasurementGro

json_temp.to_json

In [ ]:
json_temp.to_json('json_temp.json'.format(disease)

SyntaxError: incomplete input (1319822603.py, line 1)

In [ ]:
# print(json_string)
# print(json_string['FullStudiesResponse'])
# json_string['FullStudiesResponse']['FullStudies'][0]['BaselineCharacteristicsModule']['BaselinePopulationDescription']
json_string['FullStudiesResponse']['FullStudies'][0]['BaselineCharacteristicsModule']['BaselinePopulationDescription']['DemographicModule']
# json_string(['FullStudiesResponse']['FullStudies']).keys()

# print(json_string).keys()

KeyError: 'BaselineCharacteristicsModule'

In [ ]:
start = 0
end = 5
iter = 99 #number of studies to get at a time ( limited to 1000 )


for ind in check.index:
# for ind in check.index(1):
    # print(check['NCTId'][ind])

    # ct.get_full_studies(check['NCTId'][ind], max_studies=ter, min_rnk=start, fmt="json")
    json_string = ct.get_full_studies(check['NCTId'][ind], max_studies=1, min_rnk=1)
    #this one works
    print(json_string['FullStudiesResponse']['FullStudies'][0])
    # print(json_string['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['BaselineCharacteristicsModule']['BaselinePopulationDescription'])
    # print(json_string['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection'])
    # print(json_string['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['OutcomeMeasuresModule']['OutcomeMeasureList']['OutcomeMeasure'][0]['Measure'])
        # print(json_string['FullStudiesResponse']['FullStudies'][0]['Study']['ResultsSection']['DescriptionModule']['BriefSummary'])
    # print(temp['FullStudiesResponse.FullStudies'])
    # print(temp['FullStudiesResponse.FullStudies'][0]['Study']['ProtocolSection']['DesignModule']['PrimaryPurpose'])
    
    # temp2 =  pd.json_normalize(temp)
    #write to csv file
    # new_df = pd.concat([new_df, temp2])


[{'Rank': 1, 'Study': {'ProtocolSection': {'Id...


In [ ]:
new_df

In [ ]:
# for df_tmp in check[NCTId]:
#     # print(NCTI_d)
#     print(df_tmp)
from random import seed
from random import random

temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df = pd.DataFrame()
array = {}
array ['key'] = 'value'
json_data = json.dumps(array)
ct = ClinicalTrials()

start = 0
# end = count

ter = 99 #number of studies to get at a time ( limited to 1000 )

for ind in check.index:
    print(check['NCTId'][ind])
    ct.get_full_studies(check['NCTId'][ind], max_studies=ter, min_rnk=start, fmt="json")
    # ct.get_full_studies(
    #         search_expr=disease,
    #         max_studies=iter,
    #         min_rnk=i,
            
    #         # fmt="csv",
    #         )
    #     # print(count),
        # print(str(i) + "-" + str(iter) + " of " + str(count))
        # print(temp2[0])
        # temp2 =  pd.json_normalize(temp)
        # json_object = temp['FullStudiesResponse']
        # array = temp['FullStudiesResponse']
        # temp2 = pd.json_normalize(temp())
        # new_df = pd.concat([new_df, temp2])
    # print(x)
# https://clinicaltrials.gov/api/gui/demo/simple_full_study




In [ ]:
# for i in check:
#     print(i)
#     # print(NCTId)

In [ ]:

# infodf= (ct.get_full_studies(search_expr=disease, max_studies=5 ))
